# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4310029237#4088494919
date='November 8, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,12.6km/h,85.3km/h
1,Calories,2945,2945
2,Elapsed Time,5:09:14,5:09:14


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,Autopista-pradito 4.77km 171m 3.6%,20:36,13.9km/h,231W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
1,Fabricas Unidas - Glorieta Ditaires 2.05km 4...,7:30,16.5km/h,208W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
2,Ditaires - Pradito 2.68km 129m 4.8%,13:20,12.1km/h,244W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
3,Ditaires - Aragon 1.28km 52m 4.0%,5:44,13.5km/h,233W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
4,Ditaires - San Gabriel 0.71km 31m 4.3%,3:18,13.0km/h,232W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
5,ESTADIO A SAN ANTONIO DE PRADO 5.22km 301m ...,30:07,10.4km/h,239W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
6,Calle 36 # 70 A 72 Climb 2.52km 134m 5.3%,13:32,11.2km/h,246W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
7,San Gabriel-Aragón 0.35km 20m 5.7%,1:35,13.5km/h,286W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
8,Singapur-hechizo de luna 1.59km 100m 6.3%,10:03,9.5km/h,251W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."
9,Pradito - Prado 2.73km 178m 6.5%,17:48,9.2km/h,233W,"{'Calories': {'Avg': '2945', 'Max': '2945'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df.to_json(db_file)

### Save notebook here

In [9]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [10]:
!bash commit.sh

[master 997de3e] Morning Ride 6:18 AM on Sunday, December 20, 2020
 2 files changed, 494 insertions(+), 615 deletions(-)
 rewrite Strava.ipynb (61%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 2.92 KiB | 598.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   46a6802..997de3e  master -> master


In [11]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? y
